<a href="https://colab.research.google.com/github/rahiakela/interpretable-machine-learning-with-python/blob/main/2-key-concepts-of-interpretability/interpreting_cvd_epidemic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Interpreting CVD(Cardiovascular Diseases) epidemic

Imagine you are an analyst for a national health ministry, and there's a Cardiovascular Diseases (CVDs) epidemic. The minister has made it a priority to reverse the growth and reduce the case load to a 20-year low. To this end, a task force has been created to find clues in the data to ascertain the following:

- What risk factors can be addressed.
- If future cases can be predicted, interpret predictions on a case-by-case basis.

You are part of this task force!

Before we dive into the data, we must gather some important details about CVD in order to do the following:

- Understand the problem's context and relevance.
- Extract domain knowledge information that can inform our data analysis and
model interpretation.
- Relate an expert-informed background to a dataset's features.

CVDs are a group of disorders, the most common of which is coronary heart disease (also known as Ischaemic Heart Disease). According to the World Health Organization, CVD is the leading cause of death globally, killing close to 18 million people annually. Coronary heart disease and strokes (which are, for the most part, a byproduct of CVD) are the most significant contributors to that. It is estimated that 80% of CVD is made up of modifiable risk factors.

In other words, some of the preventable factors that cause CVD include
the following:

- Poor diet
- Smoking and alcohol consumption habits
- Obesity
- Lack of physical activity
- Poor sleep

Also, many of the risk factors are non-modifiable, and therefore known to be unavoidable, including the following:

- Genetic predisposition
- Old age
- Male (varies with age)

## Setup

In [1]:
!pip install --upgrade machine-learning-datasets

     |████████████████████████████████| 174kB 7.4MB/s 
     |████████████████████████████████| 51.0MB 59kB/s 
     |████████████████████████████████| 317kB 38.7MB/s 
     |████████████████████████████████| 394.5MB 33kB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 28.0MB 114kB/s 
     |████████████████████████████████| 2.9MB 25.4MB/s 
     |████████████████████████████████| 3.8MB 14.6MB/s 
     |████████████████████████████████| 471kB 46.4MB/s 
  Created wheel for pycebox: filename=pycebox-0.0.1-cp37-none-any.whl size=3640 sha256=69e872d15bc2a47509579584b2a077e4c2e839c3875d918a514d705f668bb61c
  Stored in directory: /root/.cache/pip/wheels/58/5e/65/001126ca8b1f5c71e9571d09ba579e82bb0792062e7a82e77c
  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.1.0-py2.py3-none-any.whl size=28052158 sha256=b5b3a30e39a79cd193f347a027e0eb9fbc1fa15569cc81a0ed8a72c2ea7c1cfd
  Stored in directory: /root/.cache/pip/wheels/2a/2b/0a/d6fb

In [2]:
import math
import machine_learning_datasets as mldatasets
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


##The approach

Logistic regression is one common way to rank risk factors in medical use cases. Unlike linear regression, it doesn't try to predict a continuous value for each of your observations, but it predicts a probability score that an observation belongs to a particular class. In this case, what we are trying to predict is, given $𝑥$ data for each patient, what is the $𝑦$ probability, from 0 to 1, that they have cardiovascular disease?

##Understanding and preparing the data

From this, you should be getting 70,000 records and 12 columns. We can take a peek at what was loaded.

In [3]:
cvd_df = mldatasets.load("cardiovascular-disease")
cvd_df.info()

https://raw.githubusercontent.com/caravanuden/cardio/master/cardio_train.csv downloaded to /content/data/cardio_train.csv
1 dataset files found in /content/data folder
parsing /content/data/cardio_train.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          70000 non-null  int64  
 1   gender       70000 non-null  int64  
 2   height       70000 non-null  int64  
 3   weight       70000 non-null  float64
 4   ap_hi        70000 non-null  int64  
 5   ap_lo        70000 non-null  int64  
 6   cholesterol  70000 non-null  int64  
 7   gluc         70000 non-null  int64  
 8   smoke        70000 non-null  int64  
 9   alco         70000 non-null  int64  
 10  active       70000 non-null  int64  
 11  cardio       70000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.4 MB


In [4]:
cvd_df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


###Data preparation

Age is not something we usually measure in days. In fact, for health-related predictions like this one, we might even want to bucket them into age groups since people tend to age differently.

For now, we will convert all ages into years:

In [5]:
cvd_df["age"] = cvd_df["age"] / 365.24
cvd_df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,50.358668,2,168,62.0,110,80,1,1,0,0,1,0
1,55.382762,1,156,85.0,140,90,3,1,0,0,1,1
2,51.629066,1,165,64.0,130,70,3,1,0,0,0,1
3,48.250465,2,169,82.0,150,100,1,1,0,0,1,1
4,47.842515,1,156,56.0,100,60,1,1,0,0,0,0


The result is a more understandable column because we expect age values to be between 0 and 120. We took existing data and transformed it. This is an example of feature engineering, which is when you use domain knowledge of your data to create features that better represent your problem, thereby improving your models.

There's value in performing feature engineering simply to make model outcomes more interpretable as long as this doesn't hurt model performance. As regards the age column, it can't hurt it because we haven't degraded the data. This is because you still have the decimal points for the years that represent the days.

Now we are going to take a peak at what the summary statistics are for each one of our features.

In [6]:
cvd_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,70000.0,53.304309,6.755152,29.564122,48.36272,53.945351,58.391742,64.924433
gender,70000.0,1.349571,0.476838,1.000000,1.00000,1.000000,2.000000,2.000000
height,70000.0,164.359229,8.210126,55.000000,159.00000,165.000000,170.000000,250.000000
weight,70000.0,74.205690,14.395757,10.000000,65.00000,72.000000,82.000000,200.000000
ap_hi,70000.0,128.817286,154.011419,-150.000000,120.00000,120.000000,140.000000,16020.000000
ap_lo,70000.0,96.630414,188.472530,-70.000000,80.00000,80.000000,90.000000,11000.000000
cholesterol,70000.0,1.366871,0.680250,1.000000,1.00000,1.000000,2.000000,3.000000
gluc,70000.0,1.226457,0.572270,1.000000,1.00000,1.000000,1.000000,3.000000
smoke,70000.0,0.088129,0.283484,0.000000,0.00000,0.000000,0.000000,1.000000
alco,70000.0,0.053771,0.225568,0.000000,0.00000,0.000000,0.000000,1.000000


`age` is looking good because it ranges between 29 and 65 years, which is not out of the ordinary, but there are some anomalous outliers for `ap_hi` and `ap_lo`. Blood pressure can't be negative, and the highest ever recorded was 370. These records will have to be dropped because they could lead to poor model performance and interpretability.

For good measure, we ought to make sure that `ap_hi` is always higher than `ap_lo`, so any record with that discrepancy should also be dropped.

In [7]:
cvd_df = cvd_df[(cvd_df["ap_lo"] <= 370) & (cvd_df["ap_lo"] > 0)].reset_index(drop=True)
cvd_df = cvd_df[(cvd_df["ap_hi"] <= 370) & (cvd_df["ap_hi"] > 0)].reset_index(drop=True)
cvd_df = cvd_df[cvd_df["ap_hi"] >= cvd_df["ap_lo"]].reset_index(drop=True)

Now, in order to fit a logistic regression model, we must put all objective, examination, and subjective features together as $𝑋$ and the target feature alone as $𝑦$.

In [8]:
y = cvd_df["cardio"]
X = cvd_df.drop(["cardio"], axis=1).copy()

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=2020)

print(x_train.shape, x_test.shape)

(58404, 11) (10307, 11)


##Learning about Interpretation Method Types and Scopes

Now that we have prepared our data and split it into training/test datasets, we can fit the model using the training data and print a summary of the results.

In [9]:
log_model = sm.Logit(y_train, sm.add_constant(x_train))
log_result = log_model.fit()

print(log_result.summary2())

Optimization terminated successfully.
         Current function value: 0.560389
         Iterations 6
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.191     
Dependent Variable: cardio           AIC:              65481.9016
Date:               2021-07-03 09:38 BIC:              65589.6032
No. Observations:   58404            Log-Likelihood:   -32729.   
Df Model:           11               LL-Null:          -40480.   
Df Residuals:       58392            LLR p-value:      0.0000    
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
-----------------------------------------------------------------
               Coef.   Std.Err.    z     P>|z|   [0.025   0.975] 
-----------------------------------------------------------------
const         -11.2653   0.2508 -44.9155 0.0000 -11.7569 -10.7737
age             0.0510   0.0015  34.7417 0.0000   0.0481   0.0539


The preceding summary helps us to understand which 𝑋𝑋 features contributed the most to the 𝑦𝑦 CVD diagnosis using the model coefficients (labeled Coef. in the table). Much like with linear regression, they are like a weight applied to every predictor. However, the linear combination exponent is a logistic function. This makes the interpretation more difficult.

You can only tell by looking at it that the features with the absolute highest values are `cholesterol` and `active`, but it's not very intuitive in terms of what this means.

A more interpretable way of looking at these values is revealed once you calculate the exponential of these coefficients

In [10]:
np.exp(log_result.params).sort_values(ascending=False)

cholesterol    1.652284
ap_hi          1.057969
age            1.052325
weight         1.010902
ap_lo          1.010566
gender         0.999950
height         0.996671
gluc           0.886156
smoke          0.882837
active         0.797768
alco           0.772062
const          0.000013
dtype: float64